In [ ]:
import re
import math
from pathlib import Path
from typing import List, Callable, Optional, Dict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# --- Configurações principais ---
CATEGORY_ORDER = ["Super_Weak", "Weakest", "Weak", "Strong", "Strongest"]  # ordem dos eixos no radar
PATH_COL_CANDIDATES = ["path", "filepath", "file", 0]  # tentativas de nome/posição para a primeira coluna


def _ensure_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Garante que o DataFrame tenha nomes de colunas adequados.
    A 1ª coluna (sem nome) vira 'path' se necessário.
    """
    df = df.copy()
    # Se a primeira coluna não tiver nome ou for numérica (0), renomeia para 'path'
    if df.columns[0] not in df.columns or df.columns[0] == 0:
        # Caso já tenha sido lido com header=None, defina colunas manualmente
        if len(df.columns) >= 9 and (isinstance(df.columns[0], int) or str(df.columns[0]).startswith("Unnamed")):
            df.columns = ["path", "Strongest", "Strong", "Weak", "Weakest", "Super_Weak",
                          "median_alpha", "n", "median_ntail"]
        else:
            # Se já veio com nomes, apenas garanta que a primeira seja 'path'
            cols = list(df.columns)
            cols[0] = "path"
            df.columns = cols
    else:
        # Se a primeira já tem nome, ainda assim tente padronizar para 'path' se for algo próximo
        if df.columns[0] not in ["path"]:
            cols = list(df.columns)
            cols[0] = "path"
            df.columns = cols
    return df


def _coerce_bool01(df: pd.DataFrame, cols: List[str]) -> pd.DataFrame:
    """
    Converte colunas booleanas/strings 'True'/'False' para 0/1.
    """
    df = df.copy()
    for c in cols:
        if c not in df.columns:
            raise ValueError(f"Coluna esperada não encontrada: {c}")
        # Converte strings 'True'/'False' para boolean
        if df[c].dtype == object:
            df[c] = df[c].astype(str).str.strip().str.lower().map({"true": True, "false": False})
        # Converte boolean para int
        if df[c].dtype == bool or df[c].dropna().isin([True, False]).all():
            df[c] = df[c].astype("float").astype("Int64").fillna(0).astype(int)
        # Se ainda for numérica, mantenha
        if not np.issubdtype(df[c].dtype, np.number):
            # Último recurso: tente converter direto
            df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype(int)
    return df


def default_class_extractor(path_str: str) -> str:
    """
    Extrai a 'classe' a partir do final do path do .gml.
    Exemplos:
      ..._3.gml          -> '3'
      ..._complete.gml   -> 'complete'
      ..._class_A.gml    -> 'A'  (pega o trecho entre '_' final e '.gml')
    """
    name = Path(path_str).name
    m = re.search(r"_([^_/]+)\.gml$", name)
    if m:
        return m.group(1)
    # fallback: se não encontrou, use o nome do arquivo sem extensão
    return Path(path_str).stem


def _prepare_long_table(
    dfs: List[pd.DataFrame],
    labels: Optional[List[str]] = None,
    class_extractor: Callable[[str], str] = default_class_extractor,
) -> pd.DataFrame:
    """
    Concatena os dataframes, adiciona a coluna 'source' (rótulo do DF de origem),
    extrai a 'class' do path, e retorna uma tabela longa pronta para agrupar por classe.
    """
    if labels is None:
        labels = [f"DF_{i+1}" for i in range(len(dfs))]
    assert len(labels) == len(dfs), "labels deve ter o mesmo tamanho de dfs"

    normed = []
    for df, label in zip(dfs, labels):
        df = _ensure_columns(df)
        df = _coerce_bool01(df, ["Strongest", "Strong", "Weak", "Weakest", "Super_Weak"])
        if "path" not in df.columns:
            raise ValueError("A primeira coluna deve conter o caminho do .gml e ser chamada de 'path'.")
        df = df.copy()
        df["class"] = df["path"].astype(str).apply(class_extractor)
        df["source"] = label
        normed.append(df[["path", "class", "source"] + CATEGORY_ORDER])

    long_df = pd.concat(normed, ignore_index=True)
    return long_df


def plot_radars_per_class(
    dfs: List[pd.DataFrame],
    labels: Optional[List[str]] = None,
    class_extractor: Callable[[str], str] = default_class_extractor,
    aggregate: str = "mean",  # 'mean' | 'sum' | 'first'
    figsize: tuple = (6, 6),
    tight_layout: bool = True,
    save_dir: Optional[Path] = None,
    show: bool = True,
) -> Dict[str, Path]:
    """
    Cria um radar chart por classe, sobrepondo um polígono por DataFrame (source).
    Retorna um dicionário {classe: caminho_png} se save_dir for fornecido.
    """
    long_df = _prepare_long_table(dfs, labels, class_extractor)

    # Se houver mais de uma linha por (class, source), agregue
    if aggregate == "mean":
        agg_df = long_df.groupby(["class", "source"], as_index=False)[CATEGORY_ORDER].mean()
    elif aggregate == "sum":
        agg_df = long_df.groupby(["class", "source"], as_index=False)[CATEGORY_ORDER].sum()
    elif aggregate == "first":
        agg_df = long_df.groupby(["class", "source"], as_index=False)[CATEGORY_ORDER].first()
    else:
        raise ValueError("aggregate deve ser 'mean', 'sum' ou 'first'.")

    classes = sorted(agg_df["class"].unique(), key=lambda x: (str(x) != "complete", x))
    n_axes = len(CATEGORY_ORDER)
    angles = np.linspace(0, 2 * math.pi, n_axes, endpoint=False).tolist()

    saved_paths = {}

    for cls in classes:
        sub = agg_df[agg_df["class"] == cls].sort_values("source")
        if sub.empty:
            continue

        # Preparar figura
        fig = plt.figure(figsize=figsize)
        ax = fig.add_subplot(111, polar=True)

        # eixos
        ax.set_xticks(angles)
        ax.set_xticklabels(CATEGORY_ORDER)
        ax.set_ylim(0, 1)  # como são 0/1, limite em [0,1]; se usar somas/médias além de 1, ajuste aqui.

        # plotar cada fonte
        for _, row in sub.iterrows():
            values = row[CATEGORY_ORDER].to_numpy(dtype=float)
            # fechar o polígono
            values = np.concatenate([values, values[:1]])
            angs = angles + angles[:1]
            ax.plot(angs, values, linewidth=2, label=row["source"])
            ax.fill(angs, values, alpha=0.10)

        ax.set_title(f"Scale-Free Profile — Classe: {cls}", va="bottom")
        ax.legend(loc="upper right", bbox_to_anchor=(1.25, 1.05))

        if tight_layout:
            plt.tight_layout()

        # salvar se solicitado
        if save_dir is not None:
            save_dir.mkdir(parents=True, exist_ok=True)
            out_path = save_dir / f"radar_class_{cls}.png"
            fig.savefig(out_path, dpi=200)
            saved_paths[str(cls)] = out_path

        if show:
            plt.show()
        else:
            plt.close(fig)

    return saved_paths


# -----------------------------
# EXEMPLO DE USO
# -----------------------------
if __name__ == "__main__":
    # Exemplo: lendo um CSV no formato mostrado (sem header)
    # Se você já tem os DataFrames prontos, pule esta parte e passe diretamente em dfs=[...]
    csv_text = """datasets/GenCAT/AttributedGraphDataset:BlogCatalog_n_0_m_0/gmls/AttributedGraphDataset:BlogCatalog_0.gml,False,False,False,False,False,3.150000000000002,897,261.0
datasets/GenCAT/AttributedGraphDataset:BlogCatalog_n_0_m_0/gmls/AttributedGraphDataset:BlogCatalog_1.gml,False,False,True,True,False,3.1000000000000014,771,152.0
datasets/GenCAT/AttributedGraphDataset:BlogCatalog_n_0_m_0/gmls/AttributedGraphDataset:BlogCatalog_2.gml,False,False,False,False,False,2.5600000000000014,910,399.0
datasets/GenCAT/AttributedGraphDataset:BlogCatalog_n_0_m_0/gmls/AttributedGraphDataset:BlogCatalog_3.gml,False,False,False,False,False,2.410000000000001,845,427.0
datasets/GenCAT/AttributedGraphDataset:BlogCatalog_n_0_m_0/gmls/AttributedGraphDataset:BlogCatalog_4.gml,False,False,True,True,False,3.690000000000002,898,184.0
datasets/GenCAT/AttributedGraphDataset:BlogCatalog_n_0_m_0/gmls/AttributedGraphDataset:BlogCatalog_5.gml,False,False,False,False,False,2.7700000000000014,850,328.0
datasets/GenCAT/AttributedGraphDataset:BlogCatalog_n_0_m_0/gmls/AttributedGraphDataset:BlogCatalog_complete.gml,False,False,False,False,False,1.4500000000000004,5196,5195.0
"""
    from io import StringIO
    df_example = pd.read_csv(StringIO(csv_text), header=None)
    # Suponha que você tenha outros DFs equivalentes (unatt, skymap, etc.). Aqui duplico só para demo:
    dfs = [df_example, df_example.copy()]
    labels = ["GenCAT", "OutraFonte"]

    # Cria um radar por classe (0..5 e 'complete'), sobrepondo as duas fontes
    plot_radars_per_class(
        dfs=dfs,
        labels=labels,
        aggregate="mean",          # se houver múltiplas linhas por (classe, fonte), faz a média
        figsize=(6, 6),
        save_dir=Path("radars_out"),  # opcional: salva PNGs
        show=True
    )
